In [ ]:
// Write the following CUDA program to a file named reduction_kernel.cu
%%writefile reduction_kernel.cu

// Include CUDA runtime API for memory management, kernel launches, and synchronization
#include <cuda_runtime.h>

// Include standard input/output stream library
#include <iostream>

// Include vector container from the C++ standard library
#include <vector>

// Include chrono library for performance timing
#include <chrono>

// Define number of threads per block
#define THREADS 256

// ======================
// Reduction kernel with shared memory
// ======================

// CUDA kernel that performs block-level reduction (sum) using shared memory
__global__ void reduce_sum(const float* input, float* output, int N) {

    // Declare shared memory array for partial sums within a block
    __shared__ float sdata[THREADS];

    // Thread index within the block
    int tid = threadIdx.x;

    // Global index of the element this thread will process
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Load input element into shared memory if within bounds, otherwise load 0
    sdata[tid] = (idx < N) ? input[idx] : 0.0f;

    // Synchronize all threads in the block to ensure shared memory is fully loaded
    __syncthreads();

    // Perform parallel reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {

        // Only threads with index less than s participate in this step
        if (tid < s)
            sdata[tid] += sdata[tid + s];

        // Synchronize after each reduction step
        __syncthreads();
    }

    // The first thread in each block writes the block's sum to global memory
    if (tid == 0)
        output[blockIdx.x] = sdata[0];
}

// ======================
// Host code
// ======================

// Main program entry point
int main() {

    // Total number of elements (1,048,576 elements)
    const int N = 1 << 20;

    // Create and initialize host input vector with all elements equal to 1.0
    std::vector<float> h_input(N, 1.0f);

    // Calculate the number of CUDA blocks required
    int blocks = (N + THREADS - 1) / THREADS;

    // Device pointer for input array
    float* d_input;

    // Device pointer for partial output (one value per block)
    float* d_output;

    // Allocate device memory for input array
    cudaMalloc(&d_input, N * sizeof(float));

    // Allocate device memory for partial reduction results
    cudaMalloc(&d_output, blocks * sizeof(float));

    // Copy input data from host memory to device memory
    cudaMemcpy(d_input, h_input.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    // Record start time for CUDA reduction
    auto start = std::chrono::high_resolution_clock::now();

    // Launch reduction kernel with computed number of blocks and threads
    reduce_sum<<<blocks, THREADS>>>(d_input, d_output, N);

    // Wait for kernel execution to finish
    cudaDeviceSynchronize();

    // Host vector to store partial sums from each block
    std::vector<float> h_partial(blocks);

    // Copy partial sums from device to host
    cudaMemcpy(h_partial.data(), d_output, blocks * sizeof(float), cudaMemcpyDeviceToHost);

    // Final reduction on CPU to accumulate block-level results
    float sum = 0.0f;
    for (auto v : h_partial)
        sum += v;

    // Record end time for CUDA reduction
    auto end = std::chrono::high_resolution_clock::now();

    // Print CUDA reduction result
    std::cout << "CUDA Reduction Sum: " << sum << "\n";

    // Print CUDA execution time in milliseconds
    std::cout << "Time: "
              << std::chrono::duration<double, std::milli>(end - start).count()
              << " ms\n";

    // ======================
    // CPU reduction for comparison
    // ======================

    // Record start time for CPU summation
    auto cpu_start = std::chrono::high_resolution_clock::now();

    // Perform sequential sum on CPU
    float cpu_sum = 0.0f;
    for (auto v : h_input)
        cpu_sum += v;

    // Record end time for CPU summation
    auto cpu_end = std::chrono::high_resolution_clock::now();

    // Print CPU reduction result
    std::cout << "CPU Sum: " << cpu_sum << "\n";

    // Print CPU execution time in milliseconds
    std::cout << "CPU Time: "
              << std::chrono::duration<double, std::milli>(cpu_end - cpu_start).count()
              << " ms\n";

    // Free device memory for input array
    cudaFree(d_input);

    // Free device memory for output array
    cudaFree(d_output);

    // Exit program successfully
    return 0;
}


Writing reduction_kernel.cu


In [ ]:
// Write the following CUDA program to a file named scan_kernel.cu
%%writefile scan_kernel.cu

// Include CUDA runtime API for kernel execution and memory management
#include <cuda_runtime.h>

// Include standard C++ input/output stream library
#include <iostream>

// Include vector container from the C++ standard library
#include <vector>

// Include chrono library for performance timing
#include <chrono>

// Define number of threads per CUDA block
#define THREADS 256

// ======================
// Prefix sum (exclusive scan) with shared memory
// ======================

// CUDA kernel that performs an exclusive prefix sum (scan) per block
__global__ void scan_kernel(float* data, int N) {

    // Shared memory array used to store block-local data
    __shared__ float temp[THREADS];

    // Thread index within the block
    int tid = threadIdx.x;

    // Global index of the element processed by this thread
    int idx = blockIdx.x * blockDim.x + tid;

    // Load global memory data into shared memory if within bounds
    temp[tid] = (idx < N) ? data[idx] : 0.0f;

    // Synchronize all threads to ensure shared memory is fully loaded
    __syncthreads();

    // ----------------------
    // Up-sweep (reduce) phase
    // ----------------------

    // Loop over reduction offsets, doubling each iteration
    for (int offset = 1; offset < blockDim.x; offset *= 2) {

        // Compute index for right child
        int ai = (tid + 1) * offset * 2 - 1;

        // Compute index for left child
        int bi = ai - offset;

        // Accumulate partial sums if index is within shared memory bounds
        if (ai < THREADS)
            temp[ai] += temp[bi];

        // Synchronize threads after each reduction step
        __syncthreads();
    }

    // ----------------------
    // Prepare for down-sweep
    // ----------------------

    // Clear the last element to convert inclusive scan to exclusive scan
    if (tid == 0)
        temp[THREADS - 1] = 0;

    // Synchronize to ensure last element is reset
    __syncthreads();

    // ----------------------
    // Down-sweep phase
    // ----------------------

    // Loop over offsets in reverse order
    for (int offset = THREADS / 2; offset > 0; offset /= 2) {

        // Compute index for right child
        int ai = (tid + 1) * offset * 2 - 1;

        // Compute index for left child
        int bi = ai - offset;

        // Swap and accumulate values if index is within bounds
        if (ai < THREADS) {

            // Temporarily store left value
            float t = temp[bi];

            // Move right value to left position
            temp[bi] = temp[ai];

            // Add left value to right position
            temp[ai] += t;
        }

        // Synchronize threads after each step
        __syncthreads();
    }

    // ----------------------
    // Write results back to global memory
    // ----------------------

    // Store the scanned value back to global memory if within bounds
    if (idx < N)
        data[idx] = temp[tid];
}

// ======================
// Host code
// ======================

// Main program entry point
int main() {

    // Define number of elements for the scan operation
    const int N = 1 << 16; // 65,536 elements (smaller array for scan)

    // Create and initialize host vector with all elements equal to 1.0
    std::vector<float> h_data(N, 1.0f);

    // Device pointer for data array
    float* d_data;

    // Allocate device memory for the data array
    cudaMalloc(&d_data, N * sizeof(float));

    // Copy input data from host memory to device memory
    cudaMemcpy(d_data, h_data.data(), N * sizeof(float), cudaMemcpyHostToDevice);

    // Record start time before kernel execution
    auto start = std::chrono::high_resolution_clock::now();

    // Launch scan kernel with enough blocks to cover all elements
    scan_kernel<<<(N + THREADS - 1) / THREADS, THREADS>>>(d_data, N);

    // Wait until kernel execution is complete
    cudaDeviceSynchronize();

    // Record end time after kernel execution
    auto end = std::chrono::high_resolution_clock::now();

    // Copy scanned data from device memory back to host memory
    cudaMemcpy(h_data.data(), d_data, N * sizeof(float), cudaMemcpyDeviceToHost);

    // ----------------------
    // Correctness check
    // ----------------------

    // Compute total sum on CPU for reference
    float total = 0.0f;
    for (int i = 0; i < N; i++)
        total += 1.0f;

    // Print the last element of the scanned array
    std::cout << "Last element after scan: " << h_data[N - 1] << "\n";

    // Print the expected total sum computed on CPU
    std::cout << "Total sum (CPU): " << total << "\n";

    // Print CUDA scan execution time in milliseconds
    std::cout << "CUDA Scan Time: "
              << std::chrono::duration<double, std::milli>(end - start).count()
              << " ms\n";

    // Free device memory
    cudaFree(d_data);

    // Exit program successfully
    return 0;
}


Writing scan_kernel.cu


In [ ]:
!nvcc reduction_kernel.cu -o reduction
!./reduction

CUDA Reduction Sum: 0
Time: 42.9607 ms
CPU Sum: 1.04858e+06
CPU Time: 11.673 ms


In [ ]:
!nvcc scan_kernel.cu -o scan
!./scan

Last element after scan: 1
Total sum (CPU): 65536
CUDA Scan Time: 8.79022 ms


# Control Questions – Reduction and Scan in CUDA

### 1. What is the difference between reduction and scan?

- **Reduction**  
  - Combines all elements of an array into a single value using an associative operation (e.g., sum, max, min).  
  - Example: summing all elements of an array → result is a single number.

- **Scan (Prefix Sum)**  
  - Computes all the intermediate sums (or other associative operations) of an array.  
  - Produces an array of the same size as input, where each element represents the sum of all previous elements.  
  - Example: exclusive sum of `[1, 2, 3, 4]` → `[0, 1, 3, 6]`.

---

### 2. Which CUDA memory types are used to optimize reduction and scan?

- **Shared memory**  
  - Used to store intermediate results within a block.  
  - Reduces slow accesses to global memory.  
  - Enables efficient parallel computation and synchronization among threads in a block.

- **Global memory**  
  - Used for storing the original array and final output.  
  - Access should be minimized during computation for better performance.

- **Registers / Private memory**  
  - Used for temporary variables within threads.  
  - Very fast, limited capacity.

---

### 3. How can a prefix sum be optimized on GPU?

- Use **shared memory** to hold partial sums within a block.  
- Apply **work-efficient scan algorithms**: up-sweep and down-sweep phases.  
- Use **multiple blocks** with hierarchical scan:
  1. Perform scan within each block.  
  2. Scan the block sums on the CPU or another kernel.  
  3. Add scanned block sums back to each block’s result.  
- Avoid unnecessary global memory accesses and minimize thread divergence.

---

### 4. Give an example of a problem where scan is used

- **Stream compaction** – removing unwanted elements (e.g., zeros) from an array.  
- **Histogram computation** – calculating cumulative distribution.  
- **Prefix sums in parallel algorithms** – e.g., parallel radix sort, parallel BFS, or GPU sorting algorithms.  
- **Particle simulations** – computing indices for particle reordering in physics simulations.